In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
def summarize_toucan_project(data):
    """Calculate project level statistics about Toucan retirements

    frac: fraction of all reported project retirements that have occured via Toucan
    quantity: total number of Toucan retirements
    dormancy: days since last recorded retirement, prior to Toucan.
    """
    try:
        first_touc = data.loc[data["is_touc"], "retired_at"].min()
        prior_retire = data[
            (data["retired_at"] < first_touc) & (~data["is_touc"])
        ]
        if len(prior_retire) == 0:
            days_since_prev_retire = (
                9999  # big number -- were dormant prior to toucan
            )
        else:
            most_recent_nontouc = prior_retire["retired_at"].max()
            days_since_prev_retire = (first_touc - most_recent_nontouc).days
        sums = data.groupby("is_touc").quantity.sum()
        return pd.Series(
            {
                "frac": round(sums[True] / sums.sum(), 3),
                "quantity": sums[True],
                "dormancy": days_since_prev_retire,
            }
        )
    except KeyError:  # no toucs
        return None


def munge(data):
    """Clean data for ease of use in analysis"""

    rename_d = {
        "issuanceDate": "issued_at",
        "resourceIdentifier": "id",
        "vintageStart": "vintage",
        "retireOrCancelDate": "retired_at",
        "retirementBeneficiary": "retired_by",
        "retirementDetails": "retired_note",
        "quantity": "quantity",
        "protocol": "protocol",
    }

    data = data.rename(columns=rename_d)

    data["id"] = data["id"].astype(int)
    data["vintage"] = data["vintage"].str[:4].astype(int)
    data["issued_at"] = data["issued_at"].apply(pd.Timestamp)
    data["retired_at"] = data["retired_at"].apply(pd.Timestamp)
    data["issued_yr"] = data["issued_at"].dt.year
    return data[
        [
            "id",
            "issued_at",
            "issued_yr",
            "vintage",
            "protocol",
            "retired_at",
            "retired_by",
            "retired_note",
            "quantity",
        ]
    ]


def get_project_registration_dates(df):
    """Infer project registration date

    Two inputs here: Verra transaction data and manually assembled project registration dates.
    These dates drive the Article 6 analysis.
    """
    min_issued = df.groupby("id").issued_yr.min().to_dict()

    register_dates = pd.read_csv("../data/toucan_project_register_dates.csv")
    register_dict = (
        register_dates.set_index("id")["registered_at"]
        .apply(pd.Timestamp)
        .dt.year.to_dict()
    )

    issued_criteria = {k: v for k, v in min_issued.items() if v < 2013}
    issued_criteria.update(register_dict)
    registered_at = pd.Series(issued_criteria, dtype=float)
    registered_at = registered_at.rename("registered_at")
    return registered_at

### Munge the data


In [ ]:
verra_fname = "../data/verra_2022-03-29.csv"  # you'll have to change this once you run `scripts/download_verra_data.py`!

In [ ]:
df = pd.read_csv(verra_fname)
df = munge(df)

# used in CORSIA analysis
min_vintage = df.groupby("id").vintage.min().to_dict()
df.loc[:, "min_vintage"] = df["id"].map(min_vintage)

retirements = df[~pd.isna(df["retired_at"])].copy()
retirements.loc[:, "is_touc"] = (
    retirements["retired_note"].str.startswith("TOUC").fillna(False).copy()
)

toucs = retirements[retirements["is_touc"]].copy()

# Used in Article 6 analysis
registered_at = get_project_registration_dates(df)
toucs = toucs.join(registered_at, on="id")

# Analysis


In [ ]:
touc_projects = retirements.loc[retirements["is_touc"], "id"].unique().tolist()
summary = (
    retirements[retirements["id"].isin(touc_projects)]
    .groupby("id")
    .apply(summarize_toucan_project)
    .sort_values(by=["frac", "quantity"])
)

## Inline stats

Calculation of numbers that appear in explainer text, in the order they appear!


In [ ]:
touc_total = toucs["quantity"].sum()
print(f"Total Toucan retirements: {touc_total/1_000_000:.3f}")

In [ ]:
dormant = summary[(summary["dormancy"] > 2 * 365) & (summary["frac"] <= 0.95)]
exclusive = summary[(summary["frac"] > 0.95)]

zombie_total = dormant.quantity.sum() + exclusive.quantity.sum()
print(f"Zombie credits total: {zombie_total / 1_000_000:.2f} million")

In [ ]:
frac_touc_zombie = zombie_total / touc_total
print(
    f"Percent Toucan credits classified as zombie credits: {frac_touc_zombie * 100:.2f}% "
)

In [ ]:
vcs_id = 191
project_quant = summary.loc[vcs_id, "quantity"]

print(
    f"VCS 191 is {project_quant / touc_total * 100: .2f}% of all Toucan retirements ({project_quant / 1_000_000:.2f} million credits)"
)
print(
    f"That's {summary.loc[vcs_id, 'frac'] * 100}% of all its retirements have been via Toucan"
)

### CORSIA

CORSIA has released rules on
[the types of emission units (credits) eligible for use within the program](https://www.icao.int/environmental-protection/CORSIA/Pages/CORSIA-Emissions-Units.aspx).

Each credit type has an entry describing "Eligible Unit Dates", which includes
the following bit of text:

> Issued to activities that started their first crediting period from 1 January
> 2016

This means that projects are screened by their first _crediting period_, as
opposed to project start date or the specific vintage year of credits. For each
project, we've calculated the minimum recorded vintage year within Verra's
trasaction database. So we just need to find all the Toucan credits that come
from projects with a `min_vintage` of less than 2016.


In [ ]:
corsia_cutoff = 2016
corsia_ineligible = toucs[toucs["min_vintage"] < corsia_cutoff].quantity.sum()
frac_corsia_inelig = corsia_ineligible / touc_total

print(f"CORSIA ineligible: {corsia_ineligible / 1_000_000:.2f} million")

print(
    f"Or {frac_corsia_inelig * 100:.2f}% of retired Toucan credits do not meet CORSIA criteria"
)

### Article 6

We performed a similar analysis for Article 6.

Here the eligibility criteria are less straight forward. We based our analysis
off the
[advanced version of the Article 6 rules](https://unfccc.int/documents/460950),
page 39 paragraph 75, subsection (a), which reads:

> The CDM project activity or programme of activities was registered on or after
> 1 January 2013;

Unlike CORSIA, which uses date of first crediting period, Article 6 uses project
registration date as part of its gating criteria. This introduces a slight
complication if projects are awarded retroactive credits. Specifically, what
would happen to a project that registered after 1/1/2013 but was awarded credits
for claimed mitigations that happened in 2012? A permissive reading would allow
those 2012 credits for use under Article 6.

This means we need to look at project registration dates, not vintage dates.
Unfortunately, we were unable to locate a systematic reporting of Verra project
registration dates. Instead we developed a simple method to estimate
registration date. First, if a project had an issuance event prior to 1/1/2013,
we can just use issuance date as a proxy for registration date, under the logic
that a project must be registered prior to being issued credits.  
Second, if a project has over 25,000 retirements via Toucan but an issuance date
after 1/1/2013, we downloaded the Verra project documentation to determine
registration date. We used the `Date of Issue` from the Project Design Document
as the registration date. If the project was transfered from the CDM, we used
the officially reported CDM registration date and included a link to the CDM
public database in the transcribed data. Those transcribed data are included in
this repository in `data/toucan_project_register_dates.csv`


In [ ]:
article_vi_cutoff = 2013
article_vi_ineligible = toucs[
    toucs["registered_at"] < article_vi_cutoff
].quantity.sum()
frac_article_vi_inelig = article_vi_ineligible / touc_total

print(f"Article 6 ineligible: {article_vi_ineligible / 1_000_000:.2f} million")
print(
    f"Or {frac_article_vi_inelig * 100:.2f}% of retired Toucan credits would not meet Article 6 criteria "
)

## Other stats


In [ ]:
n_zombie = len(
    set(dormant.index.unique().tolist() + exclusive.index.unique().tolist())
)
print(
    f"There are {n_zombie} zombies on the block chain, representing {n_zombie / len(touc_projects) * 100:.0f}% of all projects that have retired via Toucan"
)

## Quarterly results


In [ ]:
touc_quarterly = retirements.groupby(
    [retirements["retired_at"].dt.to_period("Q"), "is_touc"]
).quantity.sum()
all_quarterly = touc_quarterly.unstack(0).sum()

(touc_quarterly / all_quarterly).tail(5).unstack(0)

In [ ]:
toucs_ytd_2022 = retirements[
    (retirements["is_touc"]) & (retirements["retired_at"].dt.year == 2022)
].quantity.sum()
print(f"Toucan retirements YTD (2022): {toucs_ytd_2022 / 1_000_000:.2f}")

### Some dates


In [ ]:
vcs_191_date = toucs[toucs["id"] == 191].retired_at.min()
first_touc_date = toucs.retired_at.min()

## some assertions


In [ ]:
quaterly_retirements = (
    retirements.groupby(retirements["retired_at"].dt.to_period("Q"))
    .quantity.sum()
    .div(1_000_000)
    .round(1)
)
assert quaterly_retirements.loc["2021Q4"] == 47.1

In [ ]:
# random sampling and just do some asserts that we've manually checked
assert min_vintage[191] == 2006
assert min_vintage[87] == 2007
assert min_vintage[2310] == 2016
assert min_vintage[269] == 2006
assert min_vintage[2414] == 2015
assert min_vintage[1865] == 2014

# and all rows need to have a min vintage
assert len(df[pd.isna(df["min_vintage"])]) == 0